In [1]:
# To do:

# Define is the team still in it?
# - define the behaviour when in risk, and whether to double down, how long to hold, when to cut loss


In [2]:
import pandas as pd
import time
import logging
from pythonjsonlogger import jsonlogger
from flumine import FlumineSimulation, clients

In [3]:
from collections import OrderedDict, deque, defaultdict
from flumine import BaseStrategy
from flumine.order.trade import Trade
from flumine.order.order import OrderStatus
from flumine.order.ordertype import LimitOrder
from flumine.utils import get_price
from logging_setup import build_logger
import random

import time
from datetime import timedelta

class HugoStrat(BaseStrategy):
    """
    Example strateg
    """
    def __init__(self, threshold, dev, lookback, take_prof_price_diff, log_root, log_level, *a, **k):
        self.log = build_logger(log_root,log_level)  # logs/trades.log, rotated nightly
        super().__init__(*a, **k)
        self.hist = defaultdict(lambda: deque(maxlen=400))  # per runner
        self.threshold = threshold
        self.dev = dev
        self.lookback = lookback
        self.take_prof_price_diff = take_prof_price_diff
        self.startdt = None
        self._last = {}    # order_id -> last size_matched
        self.rows = []     # collected fills: [market_id, selection_id, time, size, price, side, order_id]
        self.pnl = 0.0     


    def add_market(self, market):
        self.log.info("ADD", market.market_id, market.event_name, market.event_type_id)
    
    def check_market_book(self, market, market_book):
        if market_book.status == "OPEN" and market_book.inplay:
            return True
    
    def _price_now(self, r):
        return r.last_price_traded
    
    def _price_n_secs_ago(self, key, now_dt, n=5):
        cutoff = now_dt - timedelta(seconds=n)
        dq = self.hist.get(key)
        if not dq: return None
        # find the latest sample at/before cutoff
        for t,p in reversed(dq):
            if t <= cutoff:
                return p
        return None

    def avg_back_odds(self, market, selection_id):
        total_stake, weighted = 0, 0
        for order in market.blotter:
            if order.selection_id == selection_id and order.side == "BACK":
                matched = order.size_matched
                if matched > 0:
                    total_stake += matched
                    weighted += matched * order.average_price_matched
        return weighted / total_stake if total_stake else None
    
    def process_market_book(self, market, market_book):
        if not self.startdt: self.startdt = market_book.publish_time
        elapsed = market_book.publish_time.timestamp() - self.startdt.timestamp()
        
        self.log.debug(f"process_market_book: {market.event_name}, time elapse {elapsed},  publishtime:{market_book.publish_time}")

# wait until 1 hour done then hedge out
        if elapsed > 7200:
            self.log.debug(f"elapsed time {elapsed} > 7200, hedging bets and return")
            for r in market_book.runners:
                self.hedge_selection(r)
                return
                
        now_dt = market_book.publish_time  # simulation “now”
        
        for r in market_book.runners:
            key = (market.market_id, r.selection_id)
            p = self._price_now(r)
            if p and p < self.threshold:
                self.hist[key].append((now_dt, p))
                p_5s = self._price_n_secs_ago(key, now_dt, self.lookback)
                if p_5s:
                    delta = (p-p_5s)
                    self.log.debug(f"Price update: Market, selection-id:{key} price now:{p}, odds delta:{delta}, change %: {delta / p}")
# if odds increase by X %, then bet
                    if delta>0 and (delta / p) > self.dev :
                        runner_context = self.get_runner_context(market.market_id, r.selection_id, r.handicap)
                        if runner_context.live_trade_count == 0:
                            
                            self.log.info(f"big increase odds, no live trades, placing order: {r.selection_id} 5s ago : {p_5s}  now : {p}")
                            self.log.info(f": {market.event_name}, time elapse {elapsed},  {market_book.publish_time}")
                            # back at current best lay price
                            back = get_price(r.ex.available_to_lay, 0) 
                            # create trade
                            trade = Trade(market_book.market_id, r.selection_id, r.handicap,
                                self, notes={"entry_px": back})
                            order = trade.create_order(side="BACK", order_type=LimitOrder(back, self.context["stake"]))
                            market.place_order(order)
                            self.log.info({"PLACE ORDER":market.market_id,"price":back,"event_name":market.event_name})

                            
# if profit available : hedge out at some take profit value
                av = self.avg_back_odds(market, r.selection_id)
                if av and (av - p) > self.take_prof_price_diff :
                    # total matched on backs
                    self.hedge_selection(r)

    def hedge_selection(r):        
        backs = [o for o in market.blotter if o.selection_id==r.selection_id and o.side=="BACK" and o.size_matched>0]
        stake = sum(o.size_matched for o in backs)
        best_lay = get_price(r.ex.available_to_lay, 0)
        if not best_lay: return
    
        # hedge size so Pwin == Plose
        hsize = (av*stake - stake) / best_lay
        if hsize <= 0: return
        self.log.info(f"Closing risk : runner {r.selection_id}, {hsize} @ {best_lay}")
        trade = Trade(market.market_id, r.selection_id, r.handicap, self)
        order = trade.create_order("LAY", LimitOrder(best_lay, round(hsize,2), persistence_type="LAPSE"))
        market.place_order(order)

    
# if odd slowly increase bet more
    
# if further swing in odds bet more 
            
    
    def process_orders(self, market, orders):
        # kill order if unmatched in market for greater than 2 seconds
        for order in orders:
            if order.status == OrderStatus.EXECUTABLE:
                if order.elapsed_seconds and order.elapsed_seconds > 4:
                     market.cancel_order(order)

    def process_order(self, order):
        # fully matched
        if order.status == OrderStatus.EXECUTION_COMPLETE:
            a = {"Order fully matched": order.selection_id,"avg_px":order.average_price_matched,
                 "size":order.size_matched}
            self.log.info(a)

        # partially matched
        elif order.size_matched and order.size_remaining:
            a = {"Partial match":order.selection_id,
                 "matched":order.size_matched,
                 "remaining":order.size_remaining,
                 "avg_px":order.average_price_matched}
            self.log.info(a)

    def process_closed_market(self, market, market_book):
        self.pnl = 0.0
        self.log.info(f"Processing closed market: {market.event_name}, {market.market_id}")
        for order in market.blotter:
            self.pnl += order.profit
            self.log.info(f"Order PNL {order.profit}, av size matched: {order.size_matched} av price matched: {order.average_price_matched}, date_time_created: {order.date_time_created}")
            
        self.log.warning(f"Total pnl for market:{market.event_name}, {market.market_id}, : PNL :: {self.pnl}")
            
    

In [4]:
markets = pd.read_csv("ADVeventid_marketid_AUG24.csv")["path"].tolist()

In [5]:
def run_one_trial_drawdown_optim(max_order_exposure, max_selection_exposure, stake, threshold, dev, lookback, take_prof_price_diff):
    def run_once(market, max_order_exposure, max_selection_exposure, stake, threshold, dev, lookback, take_prof_price_diff):
        client = clients.SimulatedClient()
        framework = FlumineSimulation(client=client)
    
        lists =[]
        lists.append(market)
        
        strategy = HugoStrat(   
            market_filter={"markets": lists},
            max_order_exposure=max_order_exposure,
            max_selection_exposure=max_selection_exposure,
            context={"stake": stake},
            threshold=threshold,
            dev=dev,
            lookback=lookback,
            take_prof_price_diff=take_prof_price_diff,
            log_root="./logs/backtest/",
            log_level="E")
    
        framework.add_strategy(strategy)
        framework.run()
        pnl = getattr(strategy, "pnl")
        del framework, client, strategy
        return pnl

    
    pnls = []
    for i in random.sample(markets, 45) :
        pnl = run_once(i,max_order_exposure=max_order_exposure,max_selection_exposure=max_selection_exposure, stake=stake,
                       threshold=threshold, dev=dev, lookback=lookback, take_prof_price_diff=take_prof_price_diff)
        pnls.append(pnl)
    return pnls
    

In [6]:
import optuna
from optuna.samplers import NSGAIISampler
from optuna.pruners import SuccessiveHalvingPruner

def sum_losses(equity):
    result = sum(abs(x) for x in equity if x <= 0)
    return result

def objective(trial):
    order_exp = trial.suggest_int("max_order_exposure", 5, 200, step=5)
    sel_exp   = trial.suggest_int("max_selection_exposure", 20, 200, step=10)
    stake = trial.suggest_float("stake", 1.0, 10.0, step=1)
    odds_cap = trial.suggest_float("max_odds_threshold", 1.5, 20.0)
    trig_dev = trial.suggest_float("odds_change_deviation_to_trigger_trade", 0.005, 0.20, log=True)
    lookback = trial.suggest_int("price_lookback_duration_seconds", 1, 30)
    take_prof_price_diff = trial.suggest_float("odds_change_deviation_to_trigger_take_prof", 0.02, 1, log=True)

    equity = run_one_trial_drawdown_optim(
        max_order_exposure=order_exp,
        max_selection_exposure=sel_exp,
        stake=stake,
        threshold=odds_cap,
        dev=trig_dev,
        lookback=lookback,
        take_prof_price_diff=take_prof_price_diff
    )
    
    pnl_sum = sum(equity)

    sum_loss = sum_losses(equity)
    print("pnls: ", equity, "\nsum:", pnl_sum, "sum losses : -", sum_loss)
    return pnl_sum, sum_loss

study = optuna.create_study(directions=["maximize", "minimize"],sampler=NSGAIISampler(population_size=28))
study.optimize(objective, n_trials=300, n_jobs=2)

# Choose a Pareto-optimal trial (e.g., lowest MDD above a PnL floor)
best = min(study.best_trials, key=lambda t: (t.values[1], -t.values[0]))
print("Best (PnL, MDD):", best.values)
print("Params:", best.params)


[I 2025-09-13 15:10:09,329] A new study created in memory with name: no-name-fcbed096-adf7-4d30-9545-8b8459eb0da6
[I 2025-09-13 15:12:53,820] Trial 1 finished with values: [-8.59, 14.77] and parameters: {'max_order_exposure': 190, 'max_selection_exposure': 170, 'stake': 4.0, 'max_odds_threshold': 3.212372086112194, 'odds_change_deviation_to_trigger_trade': 0.15383289837337616, 'price_lookback_duration_seconds': 6, 'odds_change_deviation_to_trigger_take_prof': 0.23890520534481374}.


pnls:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0, 0.0, 0.0, 0.0, 0.18, 0.0, 0.0, 6.0, 0.0, 0.0, -2.77, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
sum: -8.59 sum losses : - 14.77


[I 2025-09-13 15:13:36,095] Trial 0 finished with values: [134.96000000000004, 222.17] and parameters: {'max_order_exposure': 150, 'max_selection_exposure': 110, 'stake': 2.0, 'max_odds_threshold': 6.45006419759102, 'odds_change_deviation_to_trigger_trade': 0.020988675796960884, 'price_lookback_duration_seconds': 21, 'odds_change_deviation_to_trigger_take_prof': 0.19792682475731466}.


pnls:  [15.190000000000005, 0.0, -4.679999999999999, -8.72, 0.2400000000000002, -25.13, -68.28, 0.0, 6.700000000000001, 24.58, 8.32, 2.58, -2.56, 0.0, 29.090000000000003, 12.749999999999996, 0.0, 1.3800000000000001, 0.24, -0.8700000000000001, 7.049999999999997, -20.369999999999997, 19.310000000000006, -0.7799999999999998, 0.0, 1.8699999999999997, 0.0, 6.8, -6.699999999999999, 41.410000000000004, 13.08, 0.0, -50.37, 0.5499999999999992, -21.54, -5.33, 34.33999999999999, 76.26000000000002, 0.17000000000000026, 7.279999999999999, 6.57, -4.9, 28.939999999999998, -1.94, 12.43] 
sum: 134.96000000000004 sum losses : - 222.17


[I 2025-09-13 15:15:38,559] Trial 2 finished with values: [-391.06, 429.62] and parameters: {'max_order_exposure': 125, 'max_selection_exposure': 190, 'stake': 5.0, 'max_odds_threshold': 17.300379626598293, 'odds_change_deviation_to_trigger_trade': 0.05826890431498843, 'price_lookback_duration_seconds': 17, 'odds_change_deviation_to_trigger_take_prof': 0.3008082970417152}.


...

[I 2025-09-13 21:15:07,052] Trial 181 finished with values: [0.0, 0.0] and parameters: {'max_order_exposure': 145, 'max_selection_exposure': 200, 'stake': 8.0, 'max_odds_threshold': 1.610733343905984, 'odds_change_deviation_to_trigger_trade': 0.14395836662170972, 'price_lookback_duration_seconds': 1, 'odds_change_deviation_to_trigger_take_prof': 0.759176601133651}.


pnls:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
sum: 0.0 sum losses : - 0.0


[I 2025-09-13 21:16:04,299] Trial 182 finished with values: [383.0900000000001, 744.73] and parameters: {'max_order_exposure': 140, 'max_selection_exposure': 170, 'stake': 5.0, 'max_odds_threshold': 5.127954060470222, 'odds_change_deviation_to_trigger_trade': 0.007411459859034675, 'price_lookback_duration_seconds': 16, 'odds_change_deviation_to_trigger_take_prof': 0.6891485147859684}.


pnls:  [0.0, 0.0, 8.440000000000001, 97.63000000000001, -14.74, -82.13, -83.6, -20.0, 0.0, 6.45, -1.25, 18.200000000000003, 0.24, -62.48, 313.28999999999996, -107.52, 111.96000000000004, 1.76, 55.63999999999999, 0.96, 0.0, 5.67, -56.52, 0.23000000000000043, 17.369999999999997, 50.26000000000001, 157.87000000000003, 98.79, -33.739999999999995, 1.0500000000000003, -2.0299999999999976, 20.26, 76.17, -86.54000000000002, -7.770000000000003, -16.18, -18.10000000000002, 3.329999999999999, -36.14, 37.720000000000006, 26.390000000000008, -88.18999999999997, -27.8, 1.0499999999999998, 17.090000000000003] 
sum: 383.0900000000001 sum losses : - 744.73


KeyboardInterrupt: 